### Параллельное программирование

In [ ]:
# пример последовательного выполнения
print('haha')
x = 1
y = 2
print(x*y)

Блокирующие задачи  
ожидание действий. Исполнение в этот момент останавливатеся, хотя что-то можно бы и делать

In [ ]:
import threading
import time

def sleeper(n, name):
    print(f'Привет я {name}. Собираюсь поспать.')
    time.sleep(n)
    print(f'{name} проснулся')

t = threading.Thread(target=sleeper, name='Thread', args=(5, 'Thread1'))
t.start()
for i in range(10):
    print('Hello')
    if i == 8:
        t.join()
        print('Отрабатываем кода Thread1 проснеться')

In [ ]:
import threading
import time

def sleeper(n, name):
    print(f'Привет я {name}. Собираюсь поспать.')
    time.sleep(n)
    print(f'{name} проснулся')

start = time.time()
thread_list = []
for i in range(3):
    t = threading.Thread(target=sleeper, name=f'Thread_{i}', args=(5, f'Thread_{i}'))
    thread_list.append(t)
    
    t.start()
    print(f'{t.name} поток в работе')

for i in range(10):
    print('Hello')    
    
for t in thread_list:
    t.join()
    
end = time.time()
    
print(f'time taken {end-start}')
print('all operations done')

Deamons потокип

In [ ]:
import threading
import time

total = 4

def creates_item():
    global total
    for i in range(10):
        time.sleep(2)
        print('iteration {}'.format(i))
        total += 1
    print('iterations done')
    
    
def creates_item2():
    global total
    for i in range(7):
        time.sleep(1)
        print('iteration {}'.format(i))
        total += 1
    print('iterations done')
    
def limits_items():
    global total
    while True:
        if total > 5:
            print('totla overloaded')
            total -= 3
            print('subtracted by 3')
        else:
            time.sleep(1)
            print('waiting')
            
creates1 = threading.Thread(target = creates_item)
creates2 = threading.Thread(target = creates_item2)
limiter = threading.Thread(target=limits_items, daemon=True)

creates1.start()
creates2.start()
limiter.start()

creates1.join()
creates2.join()
# limiter.join()
print('')

Подклассы потоки

In [ ]:
import threading
import time

class MyThread(threading.Thread):
    def __init__(self, number, func, args):
        threading.Thread.__init__(self)
        self.number = number
        self.func   = func
        self.args   = args
        
    def run(self):
        print(f'{self.number} has started')
        self.func(*self.args)
        print(f'{self.number} has finished')
        
def double(number, cycles):
    for i in range(cycles):
        number += number
    print(number)
    
threads_list = []

for i in range(50):
    t = MyThread(number=i+1, func=double, args=[i, 3])
    threads_list.append(t)
    t.start()

for t in threads_list:
    t.join()

In [ ]:
import threading
import time

class MyThread(threading.Thread):
    # так выглядит __init__ класса thread по умолчанию
#     def __init__(self, group=None, target=None, name=None,
#                     args=(), kwargs={}, *, daemon=None):

    # создадим свое
    def __init__(self, number, style, *args, **kwargs):
        super(MyThread, self).__init__(*args, **kwargs)
        self.number = number
        self.style  = style
        
    def run(self, *args, **kwargs):
        print(f'{self.number} has started')
        super(MyThread, self).run(*args, **kwargs)
        print(f'{self.number} finished')
        
def sleeper(num, style):
    print(f'sleeping for {num} sec style {style}')
    time.sleep(num)

t = MyThread(number=3, style='green', target=sleeper,
            args=[3, 'green'])
t.start()


Болкировки

In [ ]:
import threading

l = threading.Lock() 
x = 0
COUNT = 1000

def adding_2():
    global x
    with l:
        for i in range(COUNT):
            x += 2
        
def adding_3():
    global x
    with l:
        for i in range(COUNT):
            x += 3
        
def subtracting_4():
    global x
    with l:
        for i in range(COUNT):
            x -= 4

def subtracting_1():
    global x
    with l:
        for i in range(COUNT):
            x -= 1
for i in range(1000):
    t0 = threading.Thread(target=adding_2)
    t1 = threading.Thread(target=adding_3)
    t2 = threading.Thread(target=subtracting_4)
    t3 = threading.Thread(target=subtracting_1)

    t0.start()
    t1.start()
    t2.start()
    t3.start()

    t0.join()
    t1.join()
    t2.join()
    t3.join()
    if x != 0:
        print(i,x)
        break



Очереди  
- FIFO  
- LIFO  
- Priority  

In [ ]:
import queue
import threading
import time

def putting_thread(q):
    while True:
        print('start thread')
        time.sleep(10)
        q.put(5)
        print('sup something')
        
q = queue.Queue()
t = threading.Thread(target=putting_thread, args=(q,), daemon=True)
t.start()

q.put(0)
print(q.get(), 'first item')
print('----')

print(q.get(), 'finish')

Asyncio, async/awayt

Как это работало с python 3.4

In [ ]:
import asyncio
from time import time


@asyncio.coroutine
def print_nums():
    num = 1
    while True:
        print(num)
        num +=1 
        yield from asyncio.sleep(1)

@asyncio.coroutine
def print_time():
    count = 0
    while True:
        if count % 3 == 0:
            print(f'{count} seconds have passed')
        count += 1
        yield from asyncio.sleep(1)
        

@asyncio.coroutine
def main():
    task1 = asyncio.ensure_future(print_nums())
    task2 = asyncio.ensure_future(print_time())
    
    yield from asyncio.gather(task1, task2)

    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

python 3.5

In [ ]:
import asyncio
from time import time


async def print_nums():
    num = 1
    while True:
        print(num)
        num +=1 
        await asyncio.sleep(0.5)


async def print_time():
    count = 0
    while True:
        if count % 3 == 0:
            print(f'{count} seconds have passed')
        count += 1
        await asyncio.sleep(1)
        


async def main():
    task1 = asyncio.create_task(print_nums())
    task2 = asyncio.create_task(print_time())
    
    await asyncio.gather(task1, task2)

    
asyncio.run(main())


Линейная реализация

In [ ]:
import requests


def get_file(url):
    r = requests.get(url, allow_redirects=True)
    return r

def write_file(response):
    filename = response.url.split('/')[-1]
    with open('images/' + filename, 'wb') as f:
        f.write(response.content)

def main():
    t0 = time()
    url = 'https://loremflickr.com/320/240'
    
    for i in range(10):
        write_file(get_file(url))
    
    print(time() - t0)
        
main()

In [1]:
import asyncio
import aiohttp

def write_image(data):
    filename = f'file-{int(time() * 1000)}.jpeg'
    with open('images/'+filename, 'wb') as f:
        f.write(data)

async def fetch_content(url, session):
    async with session.get(url, allow_redirect=True) as resp:
        data = await resp.read()
        write_image(data)

async def main():
    t0 = time()
    url = 'https://loremflickr.com/320/240'
    tasks = []
    async with aiohttp.ClientSession() as session:
        for i in range(10):
            task = asyncio.create_task(fetch_content(url, session))
            tasks.append(task)
        
        await asyncio.gather(*tasks)
        
asyncio.run(main())

RuntimeError: This event loop is already running